A bookstore wants to analyze book sales over a specified period to determine whether sales are increasing, decreasing, or remaining stable.

Your task is to investigate book sales for each 6-month period ending in March, April, and May of 2024. For example, when analyzing March, include the sales for March plus the preceding 5 months. The March 2024 analysis should include sales from October 1, 2023, to March 31, 2024, and similarly for April and May.

Calculate the total number of books sold for each analyzed period and identify the most popular book sold during that period.

For this task, you are provided with two datasets: one containing book sales data from 2021 to 2024, and the other containing the specific periods of time that require analysis.

### Importing libraries and data

In [95]:
import pandas as pd
import numpy as np
import os

path = r'C:\Users\LeLuu\Documents\Python_Practice\challenge_427'

book_sales = pd.read_csv(os.path.join('Book Sales.csv'), index_col=False)
periods = pd.read_csv(os.path.join('Time Periods.csv'), index_col=False)

### Data Validation

In [96]:
book_sales.head()

,Date,BookTitle
0,2024-01-01,Lord of the Flies
1,2024-01-01,1984
2,2024-01-01,Lord of the Flies
3,2024-01-01,1984
4,2024-01-01,The Great Gatsby


In [97]:
book_sales.dtypes

Date         object
BookTitle    object
dtype: object

In [98]:
book_sales.value_counts()

Date        BookTitle             
2024-02-05  The Great Gatsby          44
2024-05-23  To Kil A Mockingbird      38
2024-02-02  The Catcher in the Rye    37
2024-05-19  Crime and Punishment      37
2024-02-10  Lord of the Flies         37
                                      ..
2023-02-25  1984                       1
2022-12-04  The Corrections            1
            To Kil A Mockingbird       1
2022-12-05  Crime and Punishment       1
2022-08-03  Lord of the Flies          1
Name: count, Length: 4553, dtype: int64

In [99]:
periods.head()

,Start_YMD,End_YMD,MonthToReport
0,2023-12-01,2024-05-31,1
1,2023-11-01,2024-04-30,2
2,2023-10-01,2024-03-31,3


In [100]:
periods.dtypes

Start_YMD        object
End_YMD          object
MonthToReport     int64
dtype: object

### Solving the Challenge

In [101]:
#Convert the Date to Datetime data type
book_sales['Date'] = pd.to_datetime(book_sales['Date'])

In [102]:
#Convert both Start_YMD and End_YMD to Datetime data type
periods['Start_YMD'] = pd.to_datetime(periods['Start_YMD'])
periods['End_YMD'] = pd.to_datetime(periods['End_YMD'])

In [103]:
#Should check if Date is in the range of Start_YMD and End_YMD => use Cross Join
combine_data = pd.merge(book_sales,periods,how='cross')
combine_data

,Date,BookTitle,Start_YMD,End_YMD,MonthToReport
0,2024-01-01,Lord of the Flies,2023-12-01,2024-05-31,1
1,2024-01-01,Lord of the Flies,2023-11-01,2024-04-30,2
2,2024-01-01,Lord of the Flies,2023-10-01,2024-03-31,3
3,2024-01-01,1984,2023-12-01,2024-05-31,1
4,2024-01-01,1984,2023-11-01,2024-04-30,2
...,...,...,...,...,...
182500,2022-12-31,The Great Gatsby,2023-11-01,2024-04-30,2
182501,2022-12-31,The Great Gatsby,2023-10-01,2024-03-31,3
182502,2022-12-31,The Catcher in the Rye,2023-12-01,2024-05-31,1
182503,2022-12-31,The Catcher in the Rye,2023-11-01,2024-04-30,2


In [104]:
#Add a filter to check if Date is in Range, if true returns T otherwise returns F
combine_data['filter'] = combine_data.apply(
    lambda row: "T" if row['Start_YMD'] <= row['Date'] <= row['End_YMD'] else "F",
    axis=1
)

In [105]:
combine_data

,Date,BookTitle,Start_YMD,End_YMD,MonthToReport,filter
0,2024-01-01,Lord of the Flies,2023-12-01,2024-05-31,1,T
1,2024-01-01,Lord of the Flies,2023-11-01,2024-04-30,2,T
2,2024-01-01,Lord of the Flies,2023-10-01,2024-03-31,3,T
3,2024-01-01,1984,2023-12-01,2024-05-31,1,T
4,2024-01-01,1984,2023-11-01,2024-04-30,2,T
...,...,...,...,...,...,...
182500,2022-12-31,The Great Gatsby,2023-11-01,2024-04-30,2,F
182501,2022-12-31,The Great Gatsby,2023-10-01,2024-03-31,3,F
182502,2022-12-31,The Catcher in the Rye,2023-12-01,2024-05-31,1,F
182503,2022-12-31,The Catcher in the Rye,2023-11-01,2024-04-30,2,F


In [106]:
#Group by Start, End YMD, filter and count the filter,
in_range_filter = combine_data.groupby(by=['Start_YMD','End_YMD','filter']).agg(AllBookSalesInDateRange=('filter','count')).reset_index()

In [107]:
in_range_filter

,Start_YMD,End_YMD,filter,AllBookSalesInDateRange
0,2023-10-01,2024-03-31,F,31660
1,2023-10-01,2024-03-31,T,29175
2,2023-11-01,2024-04-30,F,30394
3,2023-11-01,2024-04-30,T,30441
4,2023-12-01,2024-05-31,F,29437
5,2023-12-01,2024-05-31,T,31398


In [108]:
#only get the T value in filter
in_range_filter = in_range_filter[in_range_filter['filter']=='T']

In [109]:
in_range_filter

,Start_YMD,End_YMD,filter,AllBookSalesInDateRange
1,2023-10-01,2024-03-31,T,29175
3,2023-11-01,2024-04-30,T,30441
5,2023-12-01,2024-05-31,T,31398


In [110]:
#count number of book sold in the date range
#group by start ymd, end ymd, filter and book title, count book title
books_sold = combine_data.groupby(by=['Start_YMD','End_YMD','filter','BookTitle']).agg(BookTitleSalesInDateRange=('BookTitle','count')).reset_index()

In [111]:
books_sold

,Start_YMD,End_YMD,filter,BookTitle,BookTitleSalesInDateRange
0,2023-10-01,2024-03-31,F,1984,3902
1,2023-10-01,2024-03-31,F,Crime and Punishment,4035
2,2023-10-01,2024-03-31,F,Lord of the Flies,4016
3,2023-10-01,2024-03-31,F,The Catcher in the Rye,3930
4,2023-10-01,2024-03-31,F,The Corrections,3959
5,2023-10-01,2024-03-31,F,The Great Gatsby,3922
6,2023-10-01,2024-03-31,F,The Lord of the Rings,3957
7,2023-10-01,2024-03-31,F,To Kil A Mockingbird,3939
8,2023-10-01,2024-03-31,T,1984,3550
9,2023-10-01,2024-03-31,T,Crime and Punishment,3683


In [112]:
#filter to return only if in range filter is T
book_sold_in_range = books_sold[books_sold['filter']=='T']

In [113]:
book_sold_in_range

,Start_YMD,End_YMD,filter,BookTitle,BookTitleSalesInDateRange
8,2023-10-01,2024-03-31,T,1984,3550
9,2023-10-01,2024-03-31,T,Crime and Punishment,3683
10,2023-10-01,2024-03-31,T,Lord of the Flies,3681
11,2023-10-01,2024-03-31,T,The Catcher in the Rye,3663
12,2023-10-01,2024-03-31,T,The Corrections,3689
13,2023-10-01,2024-03-31,T,The Great Gatsby,3685
14,2023-10-01,2024-03-31,T,The Lord of the Rings,3590
15,2023-10-01,2024-03-31,T,To Kil A Mockingbird,3634
24,2023-11-01,2024-04-30,T,1984,3695
25,2023-11-01,2024-04-30,T,Crime and Punishment,3802


In [114]:
#group by the start ymd and end ymd, find the max of booktitlesalesindaterange
highest_books = book_sold_in_range.loc[
    book_sold_in_range.groupby(['Start_YMD', 'End_YMD'])['BookTitleSalesInDateRange'].idxmax()
]

In [115]:
highest_books

,Start_YMD,End_YMD,filter,BookTitle,BookTitleSalesInDateRange
12,2023-10-01,2024-03-31,T,The Corrections,3689
29,2023-11-01,2024-04-30,T,The Great Gatsby,3873
42,2023-12-01,2024-05-31,T,Lord of the Flies,3993


In [116]:
result = pd.merge(in_range_filter,highest_books,on=['Start_YMD','End_YMD'],how='inner')[['Start_YMD','End_YMD','AllBookSalesInDateRange','BookTitle','BookTitleSalesInDateRange']]

In [117]:
result.sort_values(by='Start_YMD',ascending=False)

,Start_YMD,End_YMD,AllBookSalesInDateRange,BookTitle,BookTitleSalesInDateRange
2,2023-12-01,2024-05-31,31398,Lord of the Flies,3993
1,2023-11-01,2024-04-30,30441,The Great Gatsby,3873
0,2023-10-01,2024-03-31,29175,The Corrections,3689
